In [1]:
#COVID 19 Data Transformation/JDBC Conncetivity to Azure SQL Database/Widget creation
#Author: Pranav Thaenraj
#Date: 08/10/20

#Data from covidtracking.org
# Loding Covid data
covid_Path = "dbfs:/FileStore/tables/covid19.csv"
covid_df = spark.read.format('csv').option('inferschema', 'true').option('header', 'true').load(covid_Path)
#display(covid_df)

#convert the date column to a TimeStamp type
from pyspark.sql import functions as F 
from pyspark.sql.functions import count
covid_df = covid_df.withColumn("dateCast",covid_df['date'].cast("String"))
covid_df = covid_df.withColumn( 'Date', F.unix_timestamp(covid_df['dateCast'], 'yyyyMMdd').cast('timestamp'))

#Getting the Month only from the Time stamp
import datetime
from pyspark.sql.functions import year, month, dayofmonth,count
covid_df = covid_df.withColumn("Month", month(covid_df["Date"]))

#Getting the increase in COVID cases per month for each state
State_Positive_df = covid_df.groupBy("Month","State").sum("positiveIncrease").orderBy("Month","State").toDF("Month","State","positiveIncrease")

#Number of tests per state 
tests_df = covid_df.groupBy("State").sum("totalTestsViral").orderBy("State")
#Cleansing null data
tests_df = tests_df.na.drop().toDF("State","sum(totalTestsViral)")
#display(tests_df)

#number of cases per state that need ventilators
ventilator_df = covid_df.groupBy("State").sum("onVentilatorCurrently").orderBy("State")
covid_df.show(25)


+-------------------+-----+--------+--------+-------+---------------------+----------------------+--------------+---------------+---------------------+----------------------+---------+----------------+---------------+-------------------+-----------+-----+------------+-------------------+---------------+------------------+------------------+------------------+--------------+-------------+------------------------+---------------------+------------------+---------------------+---------------------+------------------------+---------------------------+---------------------------+-----------------------+--------------------------+-----------------+--------------------+----+----------------+----------------+-------+----------------+------------------------+-------+-------------+--------------------+--------------------+---------------+--------------------+-------------+-------------+-----+-----+--------+-----+
 Date|state|positive|negative|pending|hospitalizedCurrently|hospitalizedCumulative|inIcuCurrently|inIcuCumulative|onVentilatorCurrently|onVentilatorCumulative|recovered|dataQualityGrade| lastUpdateEt| dateModified|checkTimeEt|death|hospitalized| dateChecked|totalTestsViral|positiveTestsViral|negativeTestsViral|positiveCasesViral|deathConfirmed|deathProbable|totalTestEncountersViral|totalTestsPeopleViral|totalTestsAntibody|positiveTestsAntibody|negativeTestsAntibody|totalTestsPeopleAntibody|positiveTestsPeopleAntibody|negativeTestsPeopleAntibody|totalTestsPeopleAntigen|positiveTestsPeopleAntigen|totalTestsAntigen|positiveTestsAntigen|fips|positiveIncrease|negativeIncrease| total|totalTestResults|totalTestResultsIncrease| posNeg|deathIncrease|hospitalizedIncrease| hash|commercialScore|negativeRegularScore|negativeScore|positiveScore|score|grade|dateCast|Month|
+-------------------+-----+--------+--------+-------+---------------------+----------------------+--------------+---------------+---------------------+----------------------+---------+----------------+---------------+-------------------+-----------+-----+------------+-------------------+---------------+------------------+------------------+------------------+--------------+-------------+------------------------+---------------------+------------------+---------------------+---------------------+------------------------+---------------------------+---------------------------+-----------------------+--------------------------+-----------------+--------------------+----+----------------+----------------+-------+----------------+------------------------+-------+-------------+--------------------+--------------------+---------------+--------------------+-------------+-------------+-----+-----+--------+-----+
2020-08-15 00:00:00| AK| 4954| 291723| null| 36| null| null| null| 4| null| 1384| A|8/15/2020 00:00|2020-08-15 00:00:00|08/14 20:00| 28| null|2020-08-15 00:00:00| 296677| 4706| 291666| 4954| 28| null| null| null| null| null| null| null| null| null| null| null| null| null| 2| 85| 663| 296677| 296677| 748| 296677| 1| 0|4733694e7375f7962...| 0| 0| 0| 0| 0| null|20200815| 8|
2020-08-15 00:00:00| AL| 107580| 729437| null| 1259| 12607| null| 1313| null| 719| 41523| B|8/15/2020 11:00|2020-08-15 11:00:00|08/15 07:00| 1896| 12607|2020-08-15 11:00:00| 832794| null| null| 103357| 1828| 68| null| null| null| null| null| null| null| null| null| null| null| null| 1| 1271| 10762| 837017| 837017| 12033| 837017| 3| 151|dff25f5939be1e060...| 0| 0| 0| 0| 0| null|20200815| 8|
2020-08-15 00:00:00| AR| 51992| 536268| null| 464| 3562| null| null| 108| 460| 44905| A|8/15/2020 16:31|2020-08-15 16:31:00|08/15 12:31| 600| 3562|2020-08-15 16:31:00| 588660| null| 536268| 51992| null| null| null| null| null| null| null| null| null| null| null| null| 1836| null| 5| -400| 0| 588260| 588260| -400| 588260| 13| -4|4f931e30aed5c95a1...| 0| 0| 0| 0| 0| null|20200815| 8|
2020-08-15 00:00:00| AS| 0| 1514| null| null| null| null| null| null| null| null| C|8/15/2020 00:00|2020-08-15 00:00:00|08/14 20:00| 0| null|20

In [2]:
%scala 
Class.forName("com.microsoft.sqlserver.jdbc.SQLServerDriver")

In [3]:
#jdbc:sqlserver://azuresqlservermy.database.windows.net:1433;database=azuresqlmy;user=<username>@azuresqlservermy;password=<password>;encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30"

#Connecting to Azure SQL Server, Username and Password credentials changed for privacy
azuresqljdbcDF = ( 
         spark.read.format("jdbc")
        .option("url","jdbc:sqlserver://azuresqlservermy.database.windows.net:1433;database=azuresqlmy")
        .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver")
        .option("dbtable","test")
        .option("user", "ADMIN")
        .option("password","PASSWORD")
        .load()
       )
display (azuresqljdbcDF)

In [4]:
# write the dataframe's data to Azure SQLs database , tablename as CovidData# query 
(df.write.format("jdbc")
        .option("url","jdbc:sqlserver://azuresqlservermy.database.windows.net:1433;database=azuresqlmy")
        .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver")
        .option("dbtable","covidData")
        .option("user", "ADMIN")
        .option("password","PASSWORD")
        .save()
)

In [5]:
#Query the persisted Azure Sql's  CovidData table data with filter 
azuresqljdbcDF = ( 
         spark.read.format("jdbc")
        .option("url","jdbc:sqlserver://azuresqlservermy.database.windows.net:1433;database=azuresqlmy")
        .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver")
        .option("query","select state, positive, negative, hospitalizedCurrently from covidData where (hospitalizedCurrently > 100) ")
        .option("user", "ADMIN")
        .option("password","PASSWORD")
        .load()
       )
azuresqljdbcDF.show(5)

In [6]:
%fs ls dbfs:/FileStore/tables

path,name,size
dbfs:/FileStore/tables/2010_summary.csv,2010_summary.csv,7121
dbfs:/FileStore/tables/covid19.csv,covid19.csv,2178036
dbfs:/FileStore/tables/covid_aggregate-1.csv,covid_aggregate-1.csv,10404
dbfs:/FileStore/tables/covid_aggregate-2.csv,covid_aggregate-2.csv,18688
dbfs:/FileStore/tables/covid_aggregate-3.csv,covid_aggregate-3.csv,18688
dbfs:/FileStore/tables/covid_aggregate.csv,covid_aggregate.csv,10431
dbfs:/FileStore/tables/covid_aggregate_v1.csv,covid_aggregate_v1.csv,34169
dbfs:/FileStore/tables/data-1.xml,data-1.xml,22464
dbfs:/FileStore/tables/data.csv,data.csv,386
dbfs:/FileStore/tables/data.parquet/,data.parquet/,0


In [7]:
#load the file using dataframereader functions  
covid_proportiondata_df = (spark.read
                           .format ("csv")
                           .option("header", "true")
                           .option ("infer_schema" , "true")
                           .load("dbfs:/FileStore/tables/covid_aggregate_v1.csv")
                          ) 
# create a temp view 
covid_proportiondata_df.createOrReplaceTempView ("covid_aggregate_data")
#call the spark.sql API
spark.sql ( '''  select distinct state   from covid_aggregate_data  ''')

# show the positive, recovered , hospitalized   
spark.sql ('''select covidmonth,state,total_positive,total_death,total_hospitalized,total_recovered,statewide_positive_percentage, statewide_recovered_percentage,usa_total_recovered ,usa_total_positive,statewide_death_percentage from covid_aggregate_data order by statewide_death_percentage desc,State desc  ''' )

covid_proportiondata_df.show(25)


+----------+-----+--------------+--------------+-----------+---------------------------+------------------+---------------+------------------+------------------+---------------+-----------------------+----------------------+-------------------+-----------------------------+-----------------------------+--------------------------+---------------------------------+---------------------------------+------------------------------+
covidmonth|state|total_positive|total_negative|total_death|total_onVentilatorCurrently|total_hospitalized|total_recovered|usa_total_positive|usa_total_negative|usa_total_death|usa_total_ventcurrently|usa_total_hospitalized|usa_total_recovered|statewide_positive_percentage|statewide_negative_percentage|statewide_death_percentage|statewide_onventilator_percentage|statewide_hospitalized_percentage|statewide_recovered_percentage|
+----------+-----+--------------+--------------+-----------+---------------------------+------------------+---------------+------------------+------------------+---------------+-----------------------+----------------------+-------------------+-----------------------------+-----------------------------+--------------------------+---------------------------------+---------------------------------+------------------------------+
 April| WY| 435| 6903| 7| 0| 39| 340| 875668| 4344255| 55314| 136182| 100061| 148120| 0.05| 0.16| 0.01| 0| 0.04| 0.23|
 April| WV| 956| 39442| 40| 467| -1| 545| 875668| 4344255| 55314| 136182| 100061| 148120| 0.11| 0.91| 0.07| 0.34| 0| 0.37|
 April| WI| 6467| 52019| 300| 348| 1175| 3352| 875668| 4344255| 55314| 136182| 100061| 148120| 0.74| 1.2| 0.54| 0.26| 1.17| 2.26|
 April| WA| 7878| 111764| 539| 0| 0| 0| 875668| 4344255| 55314| 136182| 100061| 148120| 0.9| 2.57| 0.97| 0| 0| 0|
 April| VT| 559| 10408| 36| 0| -36| 15| 875668| 4344255| 55314| 136182| 100061| 148120| 0.06| 0.24| 0.07| 0| -0.04| 0.01|
 April| VI| 36| 681| 4| 0| 0| 30| 875668| 4344255| 55314| 136182| 100061| 148120| 0| 0.02| 0.01| 0| 0| 0.02|
 April| VA| 14596| 63512| 525| 5389| 3489| 2104| 875668| 4344255| 55314| 136182| 100061| 148120| 1.67| 1.46| 0.95| 3.96| 3.49| 1.42|
 April| UT| 3785| 86203| 41| 0| 317| 1939| 875668| 4344255| 55314| 136182| 100061| 148120| 0.43| 1.98| 0.07| 0| 0.32| 1.31|
 April| TX| 24821| 262487| 741| 0| 0| 13315| 875668| 4344255| 55314| 136182| 100061| 148120| 2.83| 6.04| 1.34| 0| 0| 8.99|
 April| TN| 8496| 141770| 176| 0| 870| 5201| 875668| 4344255| 55314| 136182| 100061| 148120| 0.97| 3.26| 0.32| 0| 0.87| 3.51|
 April| SD| 2341| 10970| 16| 0| 161| 1522| 875668| 4344255| 55314| 136182| 100061| 148120| 0.27| 0.25| 0.03| 0| 0.16| 1.03|
 April| SC| 5012| 45801| 222| 0| 898| 3701| 875668| 4344255| 55314| 136182| 100061| 148120| 0.57| 1.05| 0.4| 0| 0.9| 2.5|
 April| RI| 8448| 46556| 258| 998| 896| 431| 875668| 4344255| 55314| 136182| 100061| 148120| 0.96| 1.07| 0.47| 0.73| 0.9| 0.29|
 April| PR| 1300| 8118| 84| 0| 0| 0| 875668| 4344255| 55314| 136182| 100061| 148120| 0.15| 0.19| 0.15| 0| 0| 0|
 April| PA| 40920| 137957| 2229| 15890| -514| 0| 875668| 4344255| 55314| 136182| 100061| 148120| 4.67| 3.18| 4.03| 11.67| -0.51| 0|
 April| OR| 1820| 40386| 85| 1206| 424| 0| 875668| 4344255| 55314| 136182| 100061| 148120| 0.21| 0.93| 0.15| 0.89| 0.42| 0|
 April| OK| 3053| 58575| 199| 0| 526| 2401| 875668| 4344255| 55314| 136182| 100061| 148120| 0.35| 1.35| 0.36| 0| 0.53| 1.62|
 April| OH| 15828| 88871| 920| 0| 2948| 0| 875668| 4344255| 55314| 136182| 100061| 148120| 1.81| 2.05| 1.66| 0| 2.95| 0|
 April| NY| 228577| 466873| 16771| 0| 60315| 48629| 875668| 4344255| 55314| 136182| 100061| 148120| 26.1| 10.75| 30.32| 0| 60.28| 32.83|
 April| NV| 3885| 26307| 232| 0| 0| 158| 875668| 4344255| 55314| 136182| 100061| 148120| 0.44| 0.61| 0.42| 0| 0| 0.11|
 April| NM| 2932| 49626| 108| 162| 536| 708| 875668| 4344255| 55314| 136182| 100061| 148120| 0.33| 1.14| 0.2| 0.12| 0.54| 0.48|
 April| NJ| 99956| 102189| 8350| 37974| 0| 15642| 875668| 4344255| 55314| 136182| 100061| 148120| 11.41| 2.35| 15

In [8]:
#Creating widgets for covid_proportiondata_df

#Widget for the States Affected by COVID
states = covid_proportiondata_df.select("state").distinct().orderBy("State").collect()
dbutils.widgets.dropdown("Widget_CovidStates"
        ,"AK"
        ,[str(c.state) for c in states])
#Widget for the Month
covidmonths  = covid_proportiondata_df.select('covidmonth').distinct().collect()
dbutils.widgets.dropdown("Widget_Covidmonths"
        ,"August"
        ,[str(c.covidmonth) for c in covidmonths  ])

#remove any null values and use the widgets as part of the filter clause 
df2 = covid_proportiondata_df.select('total_positive','total_recovered','covidmonth','state').na.drop()
display(df2
   .filter( (df2.covidmonth == getArgument('Widget_Covidmonths')))
   .filter( df2.state == getArgument('Widget_CovidStates')  )
   #.groupBy('state' , 'covidmonth')
   #.avg())
       )


total_positive,total_recovered,covidmonth,state
2627,0,March,CO


In [9]:
#Creating Widgets for covid_df
covid_Path = "dbfs:/FileStore/tables/covid19.csv"
covid_df = spark.read.format('csv').option('inferschema', 'true').option('header', 'true').load(covid_Path)

#convert the date column to a TimeStamp type
from pyspark.sql import functions as F 
from pyspark.sql.functions import count
covid_df = covid_df.withColumn("dateCast",covid_df['date'].cast("String"))
covid_df = covid_df.withColumn( 'Date', F.unix_timestamp(covid_df['dateCast'], 'yyyyMMdd').cast('timestamp'))

#Getting the Month only from the Time stamp
import datetime
from pyspark.sql.functions import year, month, dayofmonth,count
covid_df = covid_df.withColumn("Month", month(covid_df["Date"]))
display(covid_df)

#Widget for the States Affected by COVID
states = covid_df.select("state").distinct().orderBy("State").collect()
dbutils.widgets.dropdown("States"
        ,"AK"
        ,[str(c.state) for c in states])
#Widget for the Month
covidmonths  = covid_df.select('Month').distinct().orderBy("Month").collect()
dbutils.widgets.dropdown("Month"
        ,"1"
        ,[str(c.Month) for c in covidmonths  ])

df2 = covid_df.select('positiveIncrease','recovered','Month','state').na.drop()
display(df2
   .filter( (df2.Month == getArgument('Month')))
   .filter( df2.state == getArgument('States')  )
       )


positiveIncrease,recovered,Month,state
